### Install the libraries

In [ ]:
!pip install smolagents

### Creating a Tool

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# Creating a custom tool to identify the most downloaded model on HuggingFace
from huggingface_hub import list_models

task = "text-classification"
most_downloaded_model = next(
                            iter(list_models(
                                filter = task, sort = "downloads", direction = -1
                                )
                                )
                        )
print(most_downloaded_model.id)

cross-encoder/ms-marco-MiniLM-L6-v2


### Automating the process of fetching through an agent

In [10]:
from smolagents import tool

@tool
def model_download_tool(task: str) -> str:
    """
    This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub.
    It returns the name of the checkpoint.

    Args:
        task: The task for which to get the download count.
    """
    most_downloaded_model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
    return most_downloaded_model.id

In [16]:
from smolagents import CodeAgent, InferenceClientModel

agent = CodeAgent(tools = [model_download_tool], model = InferenceClientModel())

agent.run(
    "Can you give me the name of the model that has the most downloads in the text-to-video task on HuggingFace Hub?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the model that has the most downloads in the text-to-video task on HuggingFace Hub? │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  most_downloaded_model = model_download_tool(task="text-to-video")                                                
  print(most_downloaded_model)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
QuantStack/Wan2.2-T2V-A14B-GGUF

Out: None

[Step 1: Duration 2.52 seconds| Input tokens: 2,111 | Output tokens: 57]

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  most_downloaded_model = model_download_tool(task="text-to-video")                                                
  print(most_downloaded_model)                                                                                     
  final_answer(most_downloaded_model)                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
QuantStack/Wan2.2-T2V-A14B-GGUF

Final answer: QuantStack/Wan2.2-T2V-A14B-GGUF

[Step 2: Duration 1.96 seconds| Input tokens: 4,378 | Output tokens: 140]

'QuantStack/Wan2.2-T2V-A14B-GGUF'

### Web Search Agent

In [32]:
from smolagents import CodeAgent, InferenceClientModel, WebSearchTool

model = InferenceClientModel()

web_agent = CodeAgent(
    tools = [WebSearchTool()],
    model = model,
    name = "search_agent",
    description="Runs web searches for you. Give it your query as an argument."
)

manager_agent = CodeAgent(
    tools = [], model = model, managed_agents = [web_agent]
)

manager_agent.run("Who won the Nobel Peace Prize in 2025")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Who won the Nobel Peace Prize in 2025                                                                           │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = search_agent(task="Find out who won the Nobel Peace Prize in 2025", additional_args={})                 
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────────────── New run - search_agent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'search_agent'.                                                                    │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Find out who won the Nobel Peace Prize in 2025                                                                  │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("María Corina Machado")                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: María Corina Machado

[Step 2: Duration 1.67 seconds| Input tokens: 4,792 | Output tokens: 161]

'María Corina Machado'